<a href="https://colab.research.google.com/github/lvwarren/PerceptiLabs/blob/master/SurfaceOfConvergenceForPellNumbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

def calculate_approximation(p0, p1, steps=10):
    """
    Calculates the approximation of the square root of 2.
    """
    mem = {0: p0, 1: p1}

    def P(n):
        if n in mem:
            return mem[n]
        k = 2 * P(n - 1) + P(n - 2)
        mem[n] = k
        return k

    results = []
    for n in range(1, steps + 1):
        approximation = (P(n - 1) + P(n)) / P(n)
        error = abs(math.sqrt(2) - approximation)
        results.append((n, approximation, error))
    return results

def calculate_convergence_metric(p0, p1, steps=10):
    """
    Calculates a convergence metric (lower is faster).
    """
    results = calculate_approximation(p0, p1, steps)
    return results[-1][2]

def create_surface_data(max_int, steps=10):
    """
    Generates data for the surface plot.
    """
    x_values = []
    y_values = []
    z_values = []

    for p0 in range(max_int + 1):
        for p1 in range(1, max_int + 1):
            x_values.append(p0)
            y_values.append(p1)
            z_values.append(calculate_convergence_metric(p0, p1, steps))

    return np.array(x_values), np.array(y_values), np.array(z_values)

def plot_surface(max_int, elev, azim, steps=10):
    """
    Creates the 3D surface plot.
    """
    x, y, z = create_surface_data(max_int, steps)

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Create meshgrid and interpolate
    X, Y = np.meshgrid(np.unique(x), np.unique(y))
    Z = np.zeros_like(X, dtype=float)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            dist = np.sqrt((x - X[i, j])**2 + (y - Y[i, j])**2)
            closest_indices = np.argsort(dist)[:3]
            if len(closest_indices) >= 3:
                weights = 1 / (dist[closest_indices] + 1e-10)
                Z[i, j] = np.sum(z[closest_indices] * weights) / np.sum(weights)
            else:
                Z[i, j] = z[np.argmin(dist)]

    surf = ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='k', alpha=0.8, linewidth=0.2)

    ax.set_xlabel('p0 (x)')
    ax.set_ylabel('p1 (y)')
    ax.set_zlabel('Convergence Error (z)')
    ax.set_title('Surface of Convergence to sqrt(2)')
    ax.view_init(elev=elev, azim=azim)

    plt.show()

# Create interactive sliders using ipywidgets
max_int_slider = widgets.IntSlider(value=25, min=5, max=100, step=1, description='Max Int')
elev_slider = widgets.IntSlider(value=30, min=-90, max=90, step=1, description='Elev')
azim_slider = widgets.IntSlider(value=120, min=0, max=360, step=1, description='Azim')

# Use interactive_output to connect the sliders to the plot function
interactive_plot = widgets.interactive_output(plot_surface, {
    'max_int': max_int_slider,
    'elev': elev_slider,
    'azim': azim_slider
})

# Display the sliders and the plot
display(max_int_slider, elev_slider, azim_slider, interactive_plot)

IntSlider(value=25, description='Max Int', min=5)

IntSlider(value=30, description='Elev', max=90, min=-90)

IntSlider(value=120, description='Azim', max=360)

Output()